In [1]:
# Example use of the Sonic module Sputnik.jl

See https://julialang.org, https://juliadb.org and https://juliastats.org for reference.

In [2]:
import Pkg                      # package manager
Pkg.activate("..")              # Project.toml
Pkg.instantiate()               # https://julialang.github.io/Pkg.jl/v1.0/index.html

Activating environment at `~/Documents/Calendar/Work/Noda/Code/Sonic.jl/Project.toml`


In [3]:
import Dates, JuliaDB, Random, StatsPlots

In [4]:
using StatsPlots
ENV["GKSwstype"] = "100"
gr(fmt = :png)

Plots.GRBackend()

Get secret token,

In [5]:
include("../src/LinckiiSecrets.jl")
secret = LinckiiSecrets.keab

(url = "https://canary.noda.se/~keab", key = "e86d93131e6bc12ae701d036e696cf8b")

In [6]:
include("../src/Linckii.jl")
include("../src/Sputnik.jl")

Main.Sputnik

Load site,

In [7]:
@time site = Linckii.loadsite(secret, :nodes, :devices, :sensors)
map(length, site)

 28.736444 seconds (25.10 M allocations: 1.683 GiB, 1.62% gc time)


(nodes = 279, devices = 11, sensors = 175)

Specify patterns,

In [8]:
p, q = Dates.Minute(15), Dates.Minute(4 * 4 * 15)
pp0 = rows -> Nodosus.preprocess0(rows, p, q)
pp1 = rows -> Nodosus.preprocess1(rows, p, q)
pp2 = rows -> Nodosus.preprocess2(rows, p, q)
patterns = Dict(
    4 => Dict(
        :meter_effect           => pp1,
        :meter_heatenergy       => pp2,
        :meter_primreturntemp   => pp1,
        :meter_primsupplytemp   => pp1,
        :meter_volume           => pp2,
        :meter_volumeflow       => pp1,
        :outdoortemp            => pp1,
        :outdoortemp_offset     => pp0,
    ),
    6 => Dict(
        :cloudiness             => pp1,
        :outdoortemp            => pp1,
        :wind_direction         => pp1,
        :wind_speed             => pp1,
    ),
)

Dict{Int64,Dict{Symbol,V} where V} with 2 entries:
  4 => Dict{Symbol,Function}(:meter_primreturntemp=>#9,:meter_effect=>#9,:outdo…
  6 => Dict(:cloudiness=>#9,:outdoortemp=>#9,:wind_direction=>#9,:wind_speed=>#…

Get pattern nodes,

In [13]:
rs = JuliaDB.rows(Linckii.get_pattern_nodes(patterns; site...))
if true
    rs = [(node_id = 62, device_id = 4), (node_id = 93, device_id = 4)] # [rs[Random.rand(1 : end)]]
end

2-element Array{NamedTuple{(:node_id, :device_id),Tuple{Int64,Int64}},1}:
 (node_id = 62, device_id = 4)
 (node_id = 93, device_id = 4)

Load data,

In [14]:
data = Dict()

Dict{Any,Any} with 0 entries

In [16]:
@time for r in rs
    p = Linckii.urlpath(secret)
    p = Linckii.db_path(p, "data"; db = r.node_id)
    try
        data[r.node_id] = JuliaDB.load(p)
        println("$(p): Done")
    catch e
        println("$(p): $(e)")
    end
end

db/linckii/~keab/data/62.db: Done
db/linckii/~keab/data/93.db: Done
  9.050311 seconds (9.02 M allocations: 550.692 MiB, 1.43% gc time)


## HEKA
* Primary differential temperature 
* Volume weighted primary differential temperature
* Primary return temperature
* Volume weighted primary return temperature
* Volume per energy
* Overflow
* Balance temperature 
* Domestic hot water quota
* Energy signature